In [ ]:
import pandas as pd
import os
import re
inflow=pd.read_excel('data.xlsx')
def clean_part_number(attr):
    attr_val=str(attr)
    if len(str(attr))>40:
        return ''
    else:
        return attr
def clean_catalog_number(attr):
    attr_val=str(attr)
    if ';' in attr_val:
        attr_list=attr_val.split(';')
        attr_clean_value=attr_list[0]
        return attr_clean_value
def remove_url(attr):
    attr_val=str(attr)
    if 'http' or 'www' in attr_val.lower():
        ret = re.compile(r'[http|https]*://[a-zA-Z0-9.?/&=:]*',re.S)
        rep=re.compile(r'www.[a-zA-Z0-9.?/&=:]*',re.S)
        attr_1=ret.sub("",attr_val)
        attr_2=rep.sub("",attr_1)               
        return attr_2
def iter_invalid_set(attr):
    attr_val=str(attr)
    
    for invalid in invalid_set:
        invalid = str(invalid)
        if invalid in attr_val:                
            attr_val = attr_val.replace(invalid,'')
    return attr_val
inflow['part_number.value']=inflow['part_number.value'].apply(lambda x:clean_part_number(x))
inflow['catalog_number.value']=inflow['catalog_number.value'].apply(lambda x:clean_catalog_number(x))
# inflow['rtip_product_description.value']=inflow['rtip_product_description.value'].apply(lambda x:remove_url(x))

invalid_df=pd.read_excel('invalid_character.xlsx',header=None)
invalid_values = invalid_df[0].values
invalid_set = set(invalid_values)
urlattrs=[]
for col in inflow.columns.tolist():
    if 'rtip_product_description' or 'bullet_point' in str(col):
        urlattrs.append(col)
for urlattr in urlattrs:
    inflow[urlattr]=inflow[urlattr].apply(lambda x:remove_url(x))
    inflow[urlattr]=inflow[urlattr].apply(lambda x:iter_invalid_set(x))
inflow.to_excel('inflow.xlsx',index=False)

In [49]:
import pandas as pd
import os
import re
import numpy as np
inflow=pd.read_excel('data.xlsx')
attr_list=['bullet_point','generic_keyword','subject_keyword']
# attr_list=['bullet_point','generic_keyword']
# def check_special_symbol(attr):
#     invalid_value=invalid[0].tolist()
# for attr in attr_list:
dropattr=[]
def mergeempty(inflow,attr):
    
    newattr=[]

    for col in inflow.columns.tolist():
        if str(attr) in str(col):
            newattr.append(col)
    #     if newattr=[]:
    #         pass
    #     else:
    #         return mergeempty
    def clearem(d):
        while '' in d:
            d.remove('')
        return d
    line=[]
    newattr2=newattr
    newattr2.insert(0,'asin')
    attr_inflow=inflow[newattr2].fillna('')
    for x in attr_inflow.index:
        out=clearem(attr_inflow.loc[x].tolist())
        line.append(out)
    line_df=pd.DataFrame(line)

    colus=line_df.columns.tolist()
    newname=['ASIN',attr+'.value']
    try:
        colus=colus[2:]
        for n in colus:
            n=attr+'#'+str(int(n))+'.value'
            newname.append(n)
        line_df.columns=newname
    except:
        pass
    return line_df,newattr
result_all=[]
# attr_clean=inflow[['asin']]
# attr_clean.rename(columns={"asin":"ASIN"},inplace=True)
attr_count=0
for attr in attr_list:   
    attr_count=attr_count+1
    mergeempty(inflow,attr)
    result=mergeempty(inflow,attr)[1]
    no_empty=mergeempty(inflow,attr)[0]
    if mergeempty(inflow,attr)[1]==[]:
        pass
    else:   
        result.pop(0)
        result_all.append(result)
        if attr_count==1:
            attr_clean=no_empty
        else:   
            attr_clean=pd.concat([attr_clean,no_empty],axis=1)
        result=[]
        newattr=[]

# mergeempty(inflow,'subject_keyword').to_excel(r'result_merge_list.xlsx',index=False)
attr_clean.to_excel('result_merge_list.xlsx',index=False)
# result_all.to_excel('attr_list.xlsx',index=False)